In [6]:
import warnings
warnings.filterwarnings('ignore')

# Метод наискорейшего спуска

In [1]:
import sympy as sym
import numpy as np

norm = np.linalg.norm

def get_alfa_from_x_and_gradient(f):
    # переменные
    variables = tuple(f.free_symbols)

    # сортировка переменных по числу: x1,x2,...,xn
    variables = sorted(variables, key=
                       lambda var: int(sym.srepr(var)[-3]))

    N = len(variables)

    # градиент функции по всем переменным
    gradients_f = [
        sym.diff(f, variable) for variable in variables]

    a = sym.symbols('a')

    # составляем переменные s_i для каждого x_i
    s_variables = list(sym.symbols(' '.join([
        f's{sym.srepr(variable)[-3]}' for variable in variables
    ])))

    # [(x_i, a*s_i + x_i)] - лист кортежей с заменой
    replacement = [(variables[i], variables[i] + a * s_variables[i]) 
                    for i in range(N)]

    # градиент функции по всем переменным
    gradients_f = [
        sym.diff(f, variable) for variable in variables]

    # заменяем переменные в градиенте
    gradients_f = [
        f.subs(replacement) for f in gradients_f
    ]

    # домножаем градиент по каждой переменной на s_i
    expr = sum([gradients_f[i] * s_variables[i] 
                for i in range(N)])
    
    # выражаем a
    solution = sym.solve(expr, a)[0]

    return sym.lambdify(variables + s_variables, solution)

def get_gradient(f, x0):
    # переменные
    variables = tuple(f.free_symbols)

    # сортировка переменных по числу
    variables = sorted(variables, key=
                       lambda var: int(sym.srepr(var)[-3]))

    # градиент функции по всем переменным
    gradients_f = [
        sym.diff(f, variable) for variable in variables]

    # превращение из символьного представления в функцию
    lambd_gradients_f = [
        sym.lambdify(variables, gradient_f) 
                    for gradient_f in gradients_f]
    
    grad_f_x = np.array([f(*x0) for f in lambd_gradients_f])

    return np.where(np.abs(grad_f_x) > 0.0001, grad_f_x, 0)

def steepest_descent(
    f: sym.Add,
    x0: np.ndarray,
    eps: float
):
    x1 = x0

    alfa = get_alfa_from_x_and_gradient(f)

    for k in range(1000):
        grad_f_x = get_gradient(f, x1)
        
        alfa_ = alfa(*x1, *-grad_f_x)

        x1 = x0 + alfa_ * -grad_f_x

        if norm(x1 - x0) < eps:
            return x1
        x0 = x1

array([-0.6875,  0.4375])

In [ ]:
### Дано
x1, x2 = sym.symbols('x1 x2')
# функция из методички
f = x1**2 + x1*x2 + 2*x2**2 + x1 - x2

x0 = np.array([0, 0])
eps = 0.1

steepest_descent(f, x0, eps)

In [5]:
x1, x2 = sym.symbols('x1 x2')

# функция 7 варианта
f = 10 - (x1 - 3) * sym.exp(-x1 + 3) - (x2 - 2) * sym.exp(-x1 + 2)

print(steepest_descent(f, np.array([0,0]), 0.1))

<lambdifygenerated-2009>:2: RuntimeWarning: divide by zero encountered in scalar divide
  return (-e*s1*x1 - s1*x2 + 2*s1 + 4*e*s1 + s2)/(s1*(e*s1 + s2))
C:\Users\Azerty\AppData\Local\Temp\ipykernel_12544\1023546824.py:84: RuntimeWarning: invalid value encountered in multiply
  x1 = x0 + alfa_ * -grad_f_x
<lambdifygenerated-2009>:2: RuntimeWarning: invalid value encountered in scalar multiply
  return (-e*s1*x1 - s1*x2 + 2*s1 + 4*e*s1 + s2)/(s1*(e*s1 + s2))


None
